In [1]:
from hepmc import *
import numpy as np

In [2]:
sarge = densities.Sarge(2, 4, 100.)

In [3]:
eeqq2g = densities.ee_qq_ng(2, 100., 5., .3)

In [4]:
rambo_mapping = phase_space.RamboOnDiet(100., 4)
mapped = phase_space.MappedDensity(eeqq2g, rambo_mapping)

In [5]:
%time sarge_sample = ImportanceMC(sarge)(eeqq2g.pdf, 10000)

Event 1	(avg. trials per event: 3.000000)
Event 2	(avg. trials per event: 4.500000)	XS = 138.157353 pb +- ( 64.012365 pb = 46.332941 %)
Event 3	(avg. trials per event: 7.333333)	XS = 56.839720 pb +- ( 35.984481 pb = 63.308687 %)
Event 4	(avg. trials per event: 9.250000)	XS = 44.994276 pb +- ( 20.172624 pb = 44.833756 %)
Event 5	(avg. trials per event: 9.800000)	XS = 34.144746 pb +- ( 16.935724 pb = 49.599793 %)
Event 6	(avg. trials per event: 8.333333)	XS = 33.785150 pb +- ( 17.464460 pb = 51.692711 %)
Event 7	(avg. trials per event: 7.714286)	XS = 32.844012 pb +- ( 16.357571 pb = 49.803816 %)
Event 8	(avg. trials per event: 7.625000)	XS = 40.988142 pb +- ( 16.297708 pb = 39.762007 %)
Event 9	(avg. trials per event: 9.555556)	XS = 72.504568 pb +- ( 41.417189 pb = 57.123558 %)
Event 10	(avg. trials per event: 8.900000)	XS = 72.062570 pb +- ( 40.191448 pb = 55.772987 %)
Event 20	(avg. trials per event: 16.600000)	XS = 34.064067 pb +- ( 11.896286 pb = 34.923270 %)
Event 30	(avg. trials pe

In [ ]:
densities.export_hepmc(100., sarge_sample, "../samples/qcd/2-4/sarge_weighted.hepmc")

In [ ]:
bound = sarge_sample.weights.max()

In [ ]:
%%time
# acceptance-rejection sampler
sarge_sample_unweighted = np.empty((1000, 16))
for i in range(1000):
    aprob = 0
    while np.random.random() > aprob:
        proposal = sarge.proposal()
        f = eeqq2g.pdf(proposal)
        g = sarge.proposal_pdf(None, proposal)
        aprob = f/(bound*g)
        if aprob > 1:
            print("bound too low")
    sarge_sample_unweighted[i] = proposal
sarge_sample_unweighted = Sample(data=sarge_sample_unweighted, weights=np.full(1000, 1./1000))

In [ ]:
densities.export_hepmc(100., sarge_sample_unweighted, "../samples/qcd/2-4/sarge_unweighted.hepmc")

In [ ]:
rambo_sample = PlainMC(8)(mapped, 10000)
rambo_sample.integral, rambo_sample.integral_err

In [ ]:
ys = mapped.pdf(rambo_sample.data)
max_y = ys.max()

In [ ]:
rambo = AcceptRejectSampler(mapped, max_y, 8)
%time rambo_sample = rambo.sample(1000)

In [ ]:
rambo_sample.data = rambo_mapping.map(rambo_sample.data)
rambo_sample.weights = np.full(rambo_sample.size, 1./rambo_sample.size)

In [ ]:
densities.export_hepmc(100., rambo_sample, "../samples/qcd/2-4/rambo.hepmc")

In [ ]:
# just a local sampler (rambo)
met = DefaultMetropolis(8, mapped, proposals.Gaussian(8, .01))
%time met_sample = met.sample(100000, np.random.rand(8))

In [ ]:
met_sample.data = rambo_mapping.map(met_sample.data)
met_sample.weights = np.full(met_sample.size, 1./met_sample.size)
densities.export_hepmc(100., met_sample, "../samples/qcd/2-4/metropolis.hepmc")

In [ ]:
# now sarge + local in MC3
importance = DefaultMetropolis(4 * 4, eeqq2g, proposal=sarge)  # using sarge, full momentum sample space
local = DefaultMetropolis(8, mapped, proposals.Gaussian(8, .01))  # using rambo, rambo sample space
update = MixingMarkovUpdate(8, [importance, local], target=mapped,
                            out_maps={0: rambo_mapping.map_inverse},
                            in_maps={0: rambo_mapping.map}) # rambo sample space, important to set target explicitly
%time mc3_sample = update.sample(100000, np.random.rand(8))

In [ ]:
mc3_sample.data = rambo_mapping.map(mc3_sample.data)
mc3_sample.weights = np.full(mc3_sample.size, 1./mc3_sample.size)
densities.export_hepmc(100., mc3_sample, "../samples/qcd/2-4/mc3.hepmc")